In [1]:
! pip install biopython

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
from ast import literal_eval
import unicodedata
import matplotlib.pyplot as plt
from Bio import SeqIO
pd.set_option("display.max_rows", None)
pd.set_option('display.width', 1000)

In [3]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "..", "..", "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
sys.path

['/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/datasets/generation',
 '/opt/conda/lib/python38.zip',
 '/opt/conda/lib/python3.8',
 '/opt/conda/lib/python3.8/lib-dynload',
 '',
 '/home/blessyantony/.local/lib/python3.8/site-packages',
 '/opt/conda/lib/python3.8/site-packages',
 '/opt/conda/lib/python3.8/site-packages/IPython/extensions',
 '/home/blessyantony/.ipython',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/datasets/generation/../../../../..',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/datasets/generation/../../../..',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/datasets/generation/../../..',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/datasets/generation/../..']

In [4]:
def column_stats(df, column_name, n=None):
    if n is None:
        n = df.shape[0]
    count_column_name = column_name + "_count"
    count_df = pd.DataFrame(df[column_name].value_counts()).reset_index()
    count_df.columns=[column_name, count_column_name]
    count_df[column_name + "_percent"] = count_df[count_column_name].apply(lambda x: int(x)/n*100)
    print(f"Number of unique values = {len(df[column_name].unique())}")
    print(f"{count_df}")
    return count_df

In [5]:
def print_seq_len_histogram(df, n_bins):
    sns.histplot(df["seq_len"])
    print(f"min seq len = {min(df['seq_len'])}")
    print(f"max seq len = {max(df['seq_len'])}")
    plt.show()
    
    freq, bins = np.histogram(df["seq_len"], bins=n_bins)
    n = df.shape[0]
    hist_map = []
    for i in range(n_bins):
        hist_map.append({"start": bins[i], "end":bins[i+1], "count": freq[i], "percentage": freq[i]/n*100})
    hist_df = pd.DataFrame(hist_map)
    print(hist_df)

In [33]:
def parse_uniref_fasta_file(input_file_path):
    sequences = []
    i = 0
    print("START: Parsing fasta file")
    print(input_file_path)
    # parse fasta file to extract uniref90_id, tax_id of virus/organism, and protein sequence
    with open(input_file_path) as f:
        for record in SeqIO.parse(f, "fasta"):
            i += 1
            sequences.append({"uniprot_id": record.id, "seq": str(record.seq)})

    print("END: Parsing fasta file")
    print(len(sequences))
    print(f"Number of records parsed = {i}")
    return pd.DataFrame(sequences)
    

columns = ["uniprot_id", "seq", "virus_host_name", "human_binary_label"]


In [34]:
wiv04_sequence_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/wiv04/sarscov2-S-WIV04ref.csv")
wiv04_df = pd.read_csv(wiv04_sequence_file_path)
wiv04_df.rename(columns={"uniref90_id": "uniprot_id"}, inplace=True)
wiv04_df.head()

,uniprot_id,seq,virus_name,seq_len,virus_host_name,human_binary_label
0,WIV04,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,WIV04(MN996528.1) Wuhan variant index virus,1273,homo sapiens,homo sapiens


In [28]:
input_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/uniprot/coronaviridae_s_uniprot_uniprot_metadata_embl_hosts_pruned_metadata_corrected_species_virus_host_vertebrates_w_seq_t0.01_c8.csv")
df = pd.read_csv(input_file_path)
print(df.shape)
df.head()

(3567, 12)


,uniprot_id,tax_id,uniprot_host_tax_ids,embl_ref_id,embl_host_name,virus_host_name,virus_name,virus_taxon_rank,virus_host_tax_id,virus_host_taxon_rank,seq,human_binary_label
0,A0A6G8HJ43,28295,NaN,QIM40843.1,['pig'],sus scrofa,Porcine epidemic diarrhea virus,species,9823,species,MKSLTYFWLFLPVLSTLSLPQDVTRCSANTNFRRFFSKFNVQAPAV...,NOT homo sapiens
1,A0A0G3FCN8,28295,NaN,AKJ85723.1,['swine'],sus scrofa,Porcine epidemic diarrhea virus,species,9823,species,MTPLIYFWLFLPVLLTLSLPQDVTRCQSTINFRRFFSKFNVQAPAV...,NOT homo sapiens
2,A0A8B1JDD3,2697049,[9606],QTZ33356.1,['Homo sapiens'],homo sapiens,Severe acute respiratory syndrome coronavirus 2,no rank,9606,species,SQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPF...,homo sapiens
3,A0A6G8HJ63,28295,NaN,QIM40847.1,['pig'],sus scrofa,Porcine epidemic diarrhea virus,species,9823,species,MKSLTYFWLFLPVLSTLSLPQDVTRCQSTINFRRFFSKFNVQAPAV...,NOT homo sapiens
4,A0A0H4A793,28295,NaN,AKN45969.1,['swine'],sus scrofa,Porcine epidemic diarrhea virus,species,9823,species,MKSLTYFWLFLPVLSTLSLPQDVTRCSANTNFRRFFSKFNVQAPAV...,NOT homo sapiens


In [29]:
df = pd.concat([wiv04_df, df])
print(df.shape)
df.head()

(3568, 13)


,uniprot_id,seq,virus_name,seq_len,virus_host_name,human_binary_label,tax_id,uniprot_host_tax_ids,embl_ref_id,embl_host_name,virus_taxon_rank,virus_host_tax_id,virus_host_taxon_rank
0,WIV04,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,WIV04(MN996528.1) Wuhan variant index virus,1273.0,homo sapiens,homo sapiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,A0A6G8HJ43,MKSLTYFWLFLPVLSTLSLPQDVTRCSANTNFRRFFSKFNVQAPAV...,Porcine epidemic diarrhea virus,NaN,sus scrofa,NOT homo sapiens,28295.0,NaN,QIM40843.1,['pig'],species,9823.0,species
1,A0A0G3FCN8,MTPLIYFWLFLPVLLTLSLPQDVTRCQSTINFRRFFSKFNVQAPAV...,Porcine epidemic diarrhea virus,NaN,sus scrofa,NOT homo sapiens,28295.0,NaN,AKJ85723.1,['swine'],species,9823.0,species
2,A0A8B1JDD3,SQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPF...,Severe acute respiratory syndrome coronavirus 2,NaN,homo sapiens,homo sapiens,2697049.0,[9606],QTZ33356.1,['Homo sapiens'],no rank,9606.0,species
3,A0A6G8HJ63,MKSLTYFWLFLPVLSTLSLPQDVTRCQSTINFRRFFSKFNVQAPAV...,Porcine epidemic diarrhea virus,NaN,sus scrofa,NOT homo sapiens,28295.0,NaN,QIM40847.1,['pig'],species,9823.0,species


In [30]:
unaligned_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/uniprot/aligned/coronaviridae_s_uniprot_uniprot_metadata_embl_hosts_pruned_metadata_corrected_species_virus_host_vertebrates_w_seq_t0.01_c8.csv")
df.to_csv(unaligned_file_path, index=False)

In [31]:
def get_aligned_dataset(unaligned_file_path, aligned_file_path):
    print(unaligned_file_path)
    unaligned_df = pd.read_csv(unaligned_file_path)[columns]
    print(f"unaligned_df size = {unaligned_df.shape}")
    # print(f"Adding wiv04 to unaligned dataset")
    # unaligned_df = pd.concat([unaligned_df, wiv04_df])
    # print(f"unaligned_df size = {unaligned_df.shape}")
    aligned_df = parse_uniref_fasta_file(aligned_file_path)
    print(f"aligned_df size = {aligned_df.shape}")
    aligned_df.rename(columns={"seq": "aligned_seq"}, inplace=True)
    df = aligned_df.merge(unaligned_df[columns], how="left", on="uniprot_id")
    print(f"merged df size = {df.shape}")
    return df

# Coronaviridae S Protein UniProt - EMBL mapping Aligned Dataset Generation
## Coronaviridae S Protein

### coronaviridae_s_uniprot_embl_hosts_pruned_metadata_corrected_species_virus_host_vertebrates_w_seq_t0.01_c8_aligned.fasta

In [35]:
aligned_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/uniprot/aligned/coronaviridae_s_uniprot_uniprot_metadata_embl_hosts_pruned_metadata_corrected_species_virus_host_vertebrates_w_seq_t0.01_c8_aligned.fasta")
df = get_aligned_dataset(unaligned_file_path, aligned_file_path)
df.head()

/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/datasets/generation/../../../../input/data/coronaviridae/20240313/uniprot/aligned/coronaviridae_s_uniprot_uniprot_metadata_embl_hosts_pruned_metadata_corrected_species_virus_host_vertebrates_w_seq_t0.01_c8.csv
unaligned_df size = (3568, 4)
START: Parsing fasta file
/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/datasets/generation/../../../../input/data/coronaviridae/20240313/uniprot/aligned/coronaviridae_s_uniprot_uniprot_metadata_embl_hosts_pruned_metadata_corrected_species_virus_host_vertebrates_w_seq_t0.01_c8_aligned.fasta
END: Parsing fasta file
3568
Number of records parsed = 3568
aligned_df size = (3568, 2)
merged df size = (3568, 5)


,uniprot_id,aligned_seq,seq,virus_host_name,human_binary_label
0,WIV04,-----MFVFLVLLPLVS---------------------------SQ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,homo sapiens,homo sapiens
1,P0DTC2,-----MFVFLVLLPLVS---------------------------SQ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,homo sapiens,homo sapiens
2,A0A679G9E9,-----MFVFLVLLPLVS---------------------------SQ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,homo sapiens,homo sapiens
3,A0A6M6IYS1,-----MFVFLVLLPLVS---------------------------SQ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,homo sapiens,homo sapiens
4,A0A7D5UC98,-----MFVFLVLLPLVS---------------------------SQ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,homo sapiens,homo sapiens


In [36]:
output_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/uniprot/aligned/coronaviridae_s_uniprot_uniprot_metadata_embl_hosts_pruned_metadata_corrected_species_virus_host_vertebrates_w_seq_t0.01_c8_aligned.csv")
df.to_csv(output_file_path, index=False)

In [37]:
column_stats(df, "virus_host_name")

Number of unique values = 8
       virus_host_name  virus_host_name_count  virus_host_name_percent
0           sus scrofa                   1626                45.571749
1         homo sapiens                   1492                41.816143
2          felis catus                    104                 2.914798
3  camelus dromedarius                    103                 2.886771
4           bos taurus                     83                 2.326233
5        gallus gallus                     66                 1.849776
6          canis lupus                     51                 1.429372
7        bos grunniens                     43                 1.205157


,virus_host_name,virus_host_name_count,virus_host_name_percent
0,sus scrofa,1626,45.571749
1,homo sapiens,1492,41.816143
2,felis catus,104,2.914798
3,camelus dromedarius,103,2.886771
4,bos taurus,83,2.326233
5,gallus gallus,66,1.849776
6,canis lupus,51,1.429372
7,bos grunniens,43,1.205157


## Coronaviridae S protein - SARS-CoV-2 only
### coronaviridae_s_uniref90_sars_cov_2_aligned.fasta

In [12]:
aligned_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/uniprot/sars_cov_2/coronaviridae_s_uniprot_sars_cov_2_aligned.fasta")
unaligned_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/uniprot/sars_cov_2/coronaviridae_s_uniprot_sars_cov_2.csv")
df = get_aligned_dataset(unaligned_file_path, aligned_file_path)
df.head()

/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/datasets/generation/../../../../input/data/coronaviridae/20240313/uniref/sars_cov_2/coronaviridae_s_uniref90_sars_cov_2.csv
unaligned_df size = (94, 4)
START: Parsing fasta file
/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/datasets/generation/../../../../input/data/coronaviridae/20240313/uniref/sars_cov_2/coronaviridae_s_uniref90_sars_cov_2_aligned.fasta
END: Parsing fasta file
94
Number of records parsed = 94
aligned_df size = (94, 2)
merged df size = (94, 5)


,uniref90_id,aligned_seq,seq,virus_host_name,human_binary_label
0,WIV04,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,homo sapiens,homo sapiens
1,UniRef90_A0A7U3HF24,------------------------PPAYTNSFTRGVYYPDKVFRSS...,PPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHV...,homo sapiens,homo sapiens
2,UniRef90_A0A7U3DWG5,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,homo sapiens,homo sapiens
3,UniRef90_A0A7U3EEK0,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,homo sapiens,homo sapiens
4,UniRef90_A0A7U3EEN6,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,homo sapiens,homo sapiens


In [13]:
df["seq_len"] = df["aligned_seq"].apply(lambda x: len(str(x)))

In [14]:
df["seq_len"].head()

0    1273
1    1273
2    1273
3    1273
4    1273
Name: seq_len, dtype: int64

In [15]:
aligned_csv_file_path = os.path.join(os.getcwd(), "..", "..", "..","..", "input/data/coronaviridae/20240313/uniref/sars_cov_2/coronaviridae_s_uniref90_sars_cov_2_aligned.csv")
df.to_csv(aligned_csv_file_path, index=False)